This is where I am going to write out my code for this project.

Louisville Animal Shelter Data: https://data.louisvilleky.gov/datasets/louisville-metro-ky-animal-service-intake-and-outcome/about
Nationwide Animal Shelter Data: https://www.shelteranimalscount.org/data-request-free-download/

In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Melis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
pip install pandas matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Melis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Melis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd

df = pd.read_csv('Animal_IO_Data.csv')

print(df.head())

    kennel animalid jurisdiction     intype insubtype               indate  \
0     ID08  A673162        40208      STRAY     FIELD  2021-02-21 00:00:00   
1  FREEZER  A617968        40215  OWNER SUR     FIELD  2022-12-14 00:00:00   
2      N40  A688268        40213  OWNER SUR       OTC  2021-01-19 00:00:00   
3     DW19  A688176        40258     FOSTER    RETURN  2021-01-22 00:00:00   
4   FOSTER  A688325        40218      STRAY       OTC  2021-01-21 00:00:00   

    surreason   outtype  outsubtype              outdate animaltype sex bites  \
0       STRAY       RTO   IN KENNEL  2021-02-21 00:00:00        DOG   N     N   
1       STRAY      EUTH   REQUESTED  2022-12-14 00:00:00        DOG   F     N   
2   OTHER PET  ADOPTION     WALK IN  2021-01-22 00:00:00        CAT   S     N   
3  OWNER DIED  TRANSFER  RESCUE GRP  2021-01-23 00:00:00        DOG   S     N   
4       STRAY    FOSTER         NaN  2021-01-26 00:00:00        DOG   S     N   

  petsize          color            breed so

In [6]:
dff = pd.read_excel('3 year 21-22-23 SAC Agg.xlsx')
print(dff.head())

  State  Year  Organizations  Intake - Relinquished By Owner Total-Feline  \
0    AK  2021              5                                          886   
1    AK  2022              5                                         1128   
2    AK  2023              5                                          911   
3    AL  2021             17                                         1548   
4    AL  2022             17                                         1419   

   Intake - Stray At Large Total-Feline  Intake - Transferred In Total-Feline  \
0                                  1620                                    85   
1                                  1878                                    51   
2                                  1814                                    45   
3                                  3414                                   707   
4                                  3400                                   816   

   Intake - Owner Intended Euthanasia Total-Feline